# Automatizce ve fyzice: OpenCV naposledy

Petr Čermák  
2023-10-27

## Připojte se

{{< qrcode https://cermak.science/teaching/automation/lectures/06-opencv/presentation.html >}}

[cermak.science/teaching/automation/lectures/06-opencv/06-opencv.html](https://cermak.science/teaching/automation/lectures/06-opencv/06-opencv.html)

# Free výpočetní cloudy

## Binder, Colab, Kaggle

-   free hostovaný jupyter notebook
-   stačí se zaregistrovat (pro colab google account)
-   CPU výkon srovnatelný s nb
-   dostupné GPU !!
-   nulová instalace

## Binder

-   Stáhne celý repo
-   nainstaluje requirements.txt
-   není python specific
-   založený na [repo2docker](https://github.com/jupyterhub/repo2docker)
-   [mybinder.org](https://mybinder.org/)
-   není od googlu

## Colab and kaggle

-   [colab.research.google.com](https://colab.research.google.com/)
-   jen malé rozdíly
-   ~ 12 GB RAM, ~ 100GB space, ~2x CPU
-   Kaggle obsahuje datasety, soutěže
-   Colab je rozšířenější, snadno se linkuje z repozitáře
-   přímé napojení na github

## Colab triky

-   připoj si google disk

``` python
from google.colab import drive
drive.mount('/content/gdrive')
```

-   funkční linux

``` bash
!wget url
!pip install ...
!git clone ...
```

-   nahrávání/stahování souborů

``` python
from google.colab import files
files.upload()
files.download('soubor')
```

-   procházení pandas dataframes

``` python
%load_ext google.colab.data_table
```

## GPU

-   všechny ale mají podporu GPU
    -   většinou chybí na noteboocích
    -   nutnost pro deep learning
-   vylepší výkon?
    -   nutnost kopírovat data na grafickou kartu
    -   *neexistuje* `instruction set` jako u CPU
    -   resp. každá grafika ho má jiný
    -   přístup přes API (OpenGL, DirectX, CUDA, …)
    -   velmi omezené možnosti co dělat (aby to mělo smysl)
-   Je třeba se učit CUDA?
    -   ne –\> numba

## numpy trochu interně

-   ufuncs
    -   univerzální funkce s numpy `array` po prvcích
    -   například při umocňování:

In [1]:
import numpy as np
x = np.arange(10)
print(x**2)
print(np.exp(x))

[ 0  1  4  9 16 25 36 49 64 81]
[1.00000000e+00 2.71828183e+00 7.38905610e+00 2.00855369e+01
 5.45981500e+01 1.48413159e+02 4.03428793e+02 1.09663316e+03
 2.98095799e+03 8.10308393e+03]

    - je to mnohem rychlejší

In [2]:
import math
x = np.arange(int(1e6))
%timeit np.sqrt(x)
%timeit [math.sqrt(xx) for xx in x]

1.69 ms ± 8.41 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
125 ms ± 860 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

## Numba

-   Numba umí předkompilovat pythoní kód
-   prakticky pouze numpy a cykly
    -   žádné pandas, vlasní knihovny etc.
    -   použití těchto bude delší
    -   snadno přepne na GPU

``` python
npoints = int(1e7)
a = np.arange(npoints, dtype=np.float32)

from numba import vectorize 

@vectorize
def cpu_sqrt(x):
  return math.sqrt(x)

cpu_sqrt(x)
```

``` python
import math
from numba import vectorize

@vectorize(['float32(float32)'], target='cuda')
def gpu_sqrt(x):
    return math.sqrt(x)
```

``` python
%timeit gpu_sqrt(a)
%timeit np.sqrt(a)
%timeit cpu_sqrt(a)
```

## Komplikovanější výpočet

-   Multivariabilní normálně rozdělená náhodná veličina.

``` python
points = np.random.multivariate_normal([0,0], [[1.,0.9], [0.9,1.]], 1000).astype(np.float32)
import matplotlib.pyplot as plt
plt.scatter(points[:,0], points[:,1])
```

-   převod do radiálních souřednic

``` python
theta = np.arctan2(points[:,1], points[:,0]) 
_ = plt.hist(theta, bins=100)
```

-   to samé na GPU:

``` python
@vectorize(['float32(float32, float32)'],target='cuda')
def gpu_arctan2(y, x): 
    theta = math.atan2(y,x)
    return theta

# malý trik kvůli GPU:
x = np.ascontiguousarray(points[:,0])
y = np.ascontiguousarray(points[:,1])
theta = gpu_arctan2(y, x)
plt.hist(theta, bins=200)
```

-   Porovnání

``` python
%timeit gpu_arctan2(y, x) # compiled for the GPU 
%timeit np.arctan2(y, x) # compiled for the CPU
%timeit [math.atan2(point[1], point[0]) for point in points] # plain python
```

# Detekce objektů

## Opakování

-   Potřebujeme:
    -   pozitivních data (kde jsou objekty)
    -   negativní data (kde nic není)
-   Pozitivní lze vygenerovat (např. pro logo, hrot)
    -   `opencv_createsamples`
    -   alternativně je sw jen upraví
    -   `opencv_annotation` - naklikáte objekty z obrázku
-   `opencv_traincascade`
    -   hlavní nástroj, mnoho parametrů

**Pozor:** Tyto nástroje nejsou součástí OpenCV 4, nutno stáhnout starou
verzi.

## Tooly na starší verzi

[Verze
3.4.11](https://sourceforge.net/projects/opencvlibrary/files/3.4.11/)

Stáhnout opencv-3.4.11-vc14_vc15.exe

. . .

Data k analýze:

[user.mgml.eu/automation/potato](https://user.mgml.eu/automation/potato/)